In [1]:
prefix = 'ens'
midx = '56'

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix

In [3]:
new = pd.read_csv('leak2d.csv')
new = new[new.Questionable != 1]
new.reset_index(inplace=True)
print(new.head())
print(new.shape)

ens = pd.read_csv('sub/'+prefix+midx+'.csv').fillna('')
ens = ens.set_index('Id')
print(ens.head())
print(ens.shape)

   index                                    Id Target  Questionable
0      0  94205e64-baca-11e8-b2b8-ac1f6b6435d0     16             0
1      1  53f49b94-bad3-11e8-b2b8-ac1f6b6435d0   9 10             0
2      3  47c3e666-bac6-11e8-b2b7-ac1f6b6435d0  14 25             0
3      4  05d45be8-bace-11e8-b2b8-ac1f6b6435d0   2 25             0
4      6  e5382de2-bad2-11e8-b2b8-ac1f6b6435d0   0 25             0
(272, 4)
                                     Predicted
Id                                            
00008af0-bad0-11e8-b2b8-ac1f6b6435d0         2
0000a892-bacf-11e8-b2b8-ac1f6b6435d0         5
0006faa6-bac7-11e8-b2b7-ac1f6b6435d0    0 5 25
0008baca-bad7-11e8-b2b9-ac1f6b6435d0         0
000cce7e-bad4-11e8-b2b8-ac1f6b6435d0          
(11702, 1)


In [4]:
changed = 0
unchanged = 0
for i, item in enumerate(new['Id']):
#     print(test_preds.loc[i]['Target'])
    if ens.loc[item]['Predicted'] != new.loc[i]['Target']:
        print(ens.loc[item]['Predicted'],' => ', new.loc[i]['Target'])
        changed += 1
        ens.loc[item]['Predicted'] = new.loc[i]['Target']
    else:
        unchanged += 1

print('changed',changed,' unchanged',unchanged)

print()
fname = 'sub/'+prefix+midx+'d.csv'
ens.to_csv(fname)
print(fname)

0 7  =>  16
14  =>  14 25
0 3 25  =>  2 25
0 21 25  =>  0 25
0 5  =>  0 19 21
22  =>  0 22 23
25  =>  21 25
0 19 22 25  =>  12
12 21  =>  12 21 25
4 5 25  =>  7 21
0 12 21  =>  0 21 25
12 21 25  =>  12
4  =>  4 23
0 12 21 25  =>  21
19 25  =>  0 18 25
0 17  =>  0 17 25
6 23  =>  2 23
0 21 25  =>  0 21 23
0 14 25  =>  0 14
14 16  =>  14
0 16  =>  0
0 12 16  =>  21
16 25  =>  14 16 25
25  =>  14 16 25
2  =>  1 2
0 26  =>  0
13  =>  12 13
14  =>  14 16
0 12 21 25  =>  0 12 25
4 26  =>  2 4 21 26
4 21  =>  13 21
6 25  =>  0 25
16 25  =>  14 16 25
7 21 25  =>  0 21 22
16 25  =>  25
18 19 25  =>  2 6 14 25
0 13 19  =>  13
0 5 25  =>  13
7 25  =>  21 25
0 22 25  =>  2 22
4  =>  4 21
4  =>  4 21
7  =>  2
0 16  =>  0 16 17 18
0 16  =>  0 16 17 18
0 2 11  =>  2 11
14  =>  14 16
18  =>  0 19
0 18  =>  0 19
0 18 19  =>  0 19
12  =>  12 21 25
14 16  =>  0 14
14 16  =>  0 14
1  =>  1 5
1  =>  1 25
0 4  =>  0
0 12 16  =>  21
0 16  =>  0
12 21  =>  13 21
0 14  =>  14 25
0 14  =>  14
0 14 16  =>  14
16

In [5]:
name_label_dict = {
    0:  "Nucleoplasm",  
    1:  "Nuclear membrane",   
    2:  "Nucleoli",   
    3:  "Nucleoli fibrillar center",   
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",   
    6:  "Endoplasmic reticulum",   
    7:  "Golgi apparatus",   
    8:  "Peroxisomes",   
    9:  "Endosomes",   
    10:  "Lysosomes",   
    11:  "Intermediate filaments",   
    12:  "Actin filaments",   
    13:  "Focal adhesion sites",   
    14:  "Microtubules",   
    15:  "Microtubule ends",   
    16:  "Cytokinetic bridge",   
    17:  "Mitotic spindle",   
    18:  "Microtubule organizing center",   
    19:  "Centrosome",   
    20:  "Lipid droplets",   
    21:  "Plasma membrane",   
    22:  "Cell junctions",   
    23:  "Mitochondria",   
    24:  "Aggresome",   
    25:  "Cytosol",   
    26:  "Cytoplasmic bodies",   
    27:  "Rods & rings"
}
LABEL_MAP = name_label_dict
np.set_printoptions(precision=3, suppress=True, linewidth=100)

In [6]:
# compute f1 score between two submission files
def f1_sub(csv0, csv1, num_classes=28):
    c0 = pd.read_csv(csv0)
    c1 = pd.read_csv(csv1)
    assert c0.shape == c1.shape
    s0 = [s if isinstance(s,str) else '' for s in c0.Predicted]
    s1 = [s if isinstance(s,str) else '' for s in c1.Predicted]
    p0 = [s.split() for s in s0]
    p1 = [s.split() for s in s1]
    y0 = np.zeros((c0.shape[0],num_classes)).astype(int)
    y1 = np.zeros((c0.shape[0],num_classes)).astype(int)
    # print(p0[:5])
    for i in range(c0.shape[0]):
        for j in p0[i]: y0[i,int(j)] = 1
        for j in p1[i]: y1[i,int(j)] = 1
    # print(y0[:5])
            
    return f1_score(y0, y1, average='macro')


# computute confusion matrices between two submission files
def f1_confusion(csv0, csv1, num_classes=28):
    c0 = pd.read_csv(csv0)
    c1 = pd.read_csv(csv1)
    assert c0.shape == c1.shape
    s0 = [s if isinstance(s,str) else '' for s in c0.Predicted]
    s1 = [s if isinstance(s,str) else '' for s in c1.Predicted]
    p0 = [s.split() for s in s0]
    p1 = [s.split() for s in s1]
    y0 = np.zeros((c0.shape[0],num_classes)).astype(int)
    y1 = np.zeros((c0.shape[0],num_classes)).astype(int)
    # print(p0[:5])
    for i in range(c0.shape[0]):
        for j in p0[i]: y0[i,int(j)] = 1
        for j in p1[i]: y1[i,int(j)] = 1
    # print(y0[:5])
    
    y0avg = np.average(y0,axis=0)
    y1avg = np.average(y1,axis=0)
    cm = [confusion_matrix(y0[:,i], y1[:,i]) for i in range(y0.shape[1])]
    fm = [f1_score(y0[:,i], y1[:,i]) for i in range(y0.shape[1])]
    for i in range(y0.shape[1]):
        print(LABEL_MAP[i])
        print(cm[i],' %4.2f' % fm[i],' %6.4f' % y0avg[i],' %6.4f' % y1avg[i],
             ' %6.4f' % (y0avg[i] - y1avg[i]))
        print()
#     print('y0avg')
#     print(y0avg)
#     print('y1avg')
#     print(y1avg)
#     print('y0avg - y1avg')
#     print(y0avg-y1avg)
    print('f1 macro')
    print(np.mean(fm))       
    return f1_score(y0, y1, average='macro')
 

In [7]:
f1_sub(fname,'sub/'+prefix+midx+'.csv')

0.9824802221766346

In [8]:
f1_sub(fname,'sub/ens36c.csv')

0.797164799355247

In [9]:
f1_sub(fname,'sub/ens43c.csv')

0.8221453508743345

In [10]:
f1_sub(fname,'sub/ens45c.csv')

0.8305844808609207

In [11]:
f1_sub(fname,'sub/ens53c.csv')

0.9422391339291746

In [12]:
f1_sub(fname,'sub/ens53d.csv')

0.9427982159659435

In [16]:
f1_sub(fname,'sub/ens55d.csv')

0.9468127940573596

In [13]:
f1_sub(fname,'sub/resnet15c.csv')

0.8505879318006561

In [14]:
f1_sub(fname,'sub/leak_brian_tommy_en_res34swa_re50xt_re101xtswa_wrn_4.8.csv')

0.7417081560065891

In [15]:
# f1_confusion(fname,'sub/'+prefix+midx+'c.csv')